In [1]:
import os
import time
import queue
import requests
import threading
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from keras.models import Sequential
from keras.layers.core import Dense
from keras.models import load_model

Using TensorFlow backend.


# get nba data to feed the model

In [2]:
df2 = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\2007\\East\\ATL\\home_away.xlsx')
df = pd.DataFrame(columns=df2.columns)
for i in range(2007,2020):
    mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(i)
    side = os.listdir(mypath)
    for j in side:
        mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(i) + '\\' + j
        team = os.listdir(mypath)
        for k in team:
            path = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(i) + '\\' + j + '\\' + k + '\\home_away.xlsx'
            df1 = pd.read_excel(path)
            df = df.append(df1)

In [3]:
df

,Date,Tm,place,Opp,FG,FGA,FG%,3P,3PA,3P%,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,result
0,2007-02-25,ATL,away,PHO,0.0,4.0,1.351,-4.0,-3.0,-1.167,...,8.0,-5.0,-3.0,4.0,1.0,12.0,-9.0,-15.7,-90.0,0.0
1,2007-02-26,ATL,home,DAL,-10.0,-15.0,-1.097,-6.0,-11.0,-2.250,...,-3.0,-9.0,-3.0,-1.0,9.0,2.0,-23.0,-37.6,-230.0,0.0
2,2007-02-28,ATL,home,NOK,5.0,-5.0,0.930,1.0,5.0,0.866,...,-14.0,5.0,-4.0,4.0,2.0,13.0,-7.0,-13.0,-70.0,0.0
3,2007-03-02,ATL,home,WAS,-7.0,-11.0,-2.012,-1.0,0.0,-0.047,...,-4.0,-3.0,1.0,3.0,1.0,0.0,-1.0,2.3,-10.0,0.0
4,2007-03-03,ATL,away,NYK,-3.0,-11.0,-0.584,-2.0,-8.0,0.629,...,1.0,2.0,-4.0,4.0,8.0,-2.0,-4.0,-3.0,-40.0,0.0
5,2007-03-05,ATL,home,MIA,2.0,9.0,-0.781,-4.0,-6.0,-1.154,...,0.0,-7.0,-4.0,2.0,3.0,13.0,-7.0,-23.4,-70.0,0.0
6,2007-03-07,ATL,away,WAS,-6.0,-16.0,-0.834,-2.0,-9.0,-0.138,...,3.0,0.0,-5.0,5.0,-2.0,-9.0,3.0,14.4,30.0,1.0
7,2007-03-09,ATL,away,MEM,-3.0,-2.0,0.285,-2.0,-5.0,0.708,...,-6.0,-2.0,2.0,-3.0,-1.0,-6.0,1.0,-0.3,10.0,1.0
8,2007-03-10,ATL,away,MIN,0.0,3.0,-1.003,0.0,1.0,0.233,...,5.0,0.0,5.0,-2.0,1.0,0.0,6.0,10.8,60.0,1.0
9,2007-03-13,ATL,away,PHI,6.0,2.0,-1.206,0.0,2.0,0.167,...,14.0,7.0,-4.0,-2.0,2.0,4.0,12.0,14.3,120.0,1.0


In [3]:
df = df.reset_index(drop=True)
dataX = df.drop(['Date', 'Tm', 'place', 'Opp', 'result', '+/-', 'GmSc'], axis=1)
dataY = df['result']

# split data to train_x, train_y, test_x, test_y

In [4]:
train_x = np.array(dataX)[::2]
train_y = np.array(dataY)[::2] 
test_x = np.array(dataX)[1::2]
test_y = np.array(dataY)[1::2]
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((13289, 18), (13289,), (13289, 18), (13289,))

# build Sequential model

In [18]:
model = Sequential() 
model.add(Dense(30, input_dim=train_x.shape[1], activation='relu')) 
model.add(Dense(20, activation='relu')) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 30)                570       
_________________________________________________________________
dense_5 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 21        
Total params: 1,211
Trainable params: 1,211
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(train_x, train_y, batch_size=16, epochs=10)

Epoch 1/10
13289/13289 [==============================] - 3s 204us/step - loss: 0.1343 - acc: 0.9485
Epoch 2/10
13289/13289 [==============================] - 2s 139us/step - loss: 0.0707 - acc: 0.9760
Epoch 3/10
13289/13289 [==============================] - 2s 138us/step - loss: 0.0562 - acc: 0.9801
Epoch 4/10
13289/13289 [==============================] - 2s 138us/step - loss: 0.0483 - acc: 0.9832
Epoch 5/10
13289/13289 [==============================] - 2s 150us/step - loss: 0.0421 - acc: 0.9854
Epoch 6/10
13289/13289 [==============================] - 2s 156us/step - loss: 0.0368 - acc: 0.9872
Epoch 7/10
13289/13289 [==============================] - 2s 163us/step - loss: 0.0355 - acc: 0.9877
Epoch 8/10
13289/13289 [==============================] - 2s 163us/step - loss: 0.0315 - acc: 0.9897
Epoch 9/10
13289/13289 [==============================] - 2s 163us/step - loss: 0.0298 - acc: 0.9907
Epoch 10/10
13289/13289 [==============================] - 2s 167us/step - loss: 0.0301 - a

# test the model result correct rate by using validation data

In [21]:
model.evaluate(test_x, test_y)

13289/13289 [==============================] - 1s 73us/step


[0.045742467252213356, 0.98818571751072315]

# save the model

In [15]:
model.save("nba-model.hdf5") 

# load the model I saved and test it

In [21]:
model1 = load_model('nba-model.hdf5')
print('test after load: ', model1.evaluate(test_x, test_y))

12059/12059 [==============================] - 1s 58us/step
test after load:  [0.040774939581462208, 0.9886391906459906]


# predict function

In [9]:
def predict(home, away):
    home_data = df.loc[(df.Tm == home) & (df.place == 'home'),].tail(5).mean()
    away_data = df.loc[(df.Tm == away) & (df.place == 'away'),].tail(5).mean()
    home_data = home_data.drop(['result', '+/-', 'GmSc'], axis=0)
    away_data = away_data.drop(['result', '+/-', 'GmSc'], axis=0)
    new_x = np.array(home_data - away_data)
    model1 = load_model('nba-model.hdf5')
    return model1.predict_classes(new_x[np.newaxis,:])[0][0]

# get all games in one day and predict

In [4]:
def predict(home, away):
    home_data = df.loc[(df.Tm == home) & (df.place == 'home'),].tail(5).mean()
    away_data = df.loc[(df.Tm == away) & (df.place == 'away'),].tail(5).mean()
    home_data = home_data.drop(['result', '+/-', 'GmSc'], axis=0)
    away_data = away_data.drop(['result', '+/-', 'GmSc'], axis=0)
    new_x = np.array(home_data - away_data)
    model1 = load_model('nba-model.hdf5')
    return model1.predict_classes(new_x[np.newaxis,:])[0][0]

month = ['october', 'november', 'december', 'january', 'february', 'march', 'april']
path = 'C:\\Users\\ecom\\Desktop\\NBAData\\schedual'
for y in range(2018,2019):
    oneyear = str(y)
    for m in month:
        path = 'C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + oneyear + '\\' + m + '.xlsx'

        # dfp = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\2017\\february.xlsx')
        dfp = pd.read_excel(path)
        date1 = dfp['Date'].values
        dateuni = np.unique(date1)
        for gamecount in range(len(dateuni)):
            # 選擇2017賽季當月的第幾天
            dfdate = dfp.loc[dfp.Date == dateuni[gamecount],]
            com = []
            for i in range(len(dfdate)):
                com1 = [dfdate.iloc[i,3], dfdate.iloc[i,1]]
                com.append(com1)
            count = 0
            for i in range(len(com)):
                p = predict(com[i][0], com[i][1])
                if p == 1:
                    if dfdate.iloc[i,4] > dfdate.iloc[i,2]:
                        count = count + 1
                    print('%s( win) vs %s(loss)' % (com[i][0], com[i][1]))
                else:
                    if dfdate.iloc[i,2] > dfdate.iloc[i,4]:
                        count = count + 1
                    print('%s(loss) vs %s( win)' % (com[i][0], com[i][1]))

            games = len(com)
            print(count/games)

CLE( win) vs NYK(loss)
GSW(loss) vs SAS( win)
POR(loss) vs UTA( win)
0.6666666666666666
BOS(loss) vs BRK( win)
IND(loss) vs DAL( win)
LAL(loss) vs HOU( win)
MEM(loss) vs MIN( win)
MIL(loss) vs CHO( win)
NOP(loss) vs DEN( win)
ORL(loss) vs MIA( win)
PHI(loss) vs OKC( win)
PHO(loss) vs SAC( win)
TOR( win) vs DET(loss)
0.6
ATL(loss) vs WAS( win)
CHI(loss) vs BOS( win)
POR(loss) vs LAC( win)
SAC(loss) vs SAS( win)
0.5
BRK(loss) vs IND( win)
DAL( win) vs HOU(loss)
DET(loss) vs ORL( win)
MIA(loss) vs CHO( win)
NOP(loss) vs GSW( win)
OKC( win) vs PHO(loss)
TOR(loss) vs CLE( win)
UTA( win) vs LAL(loss)
0.625
CHI(loss) vs IND( win)
CHO(loss) vs BOS( win)
CLE(loss) vs ORL( win)
DEN(loss) vs POR( win)
MIL(loss) vs BRK( win)
NYK(loss) vs MEM( win)
PHI(loss) vs ATL( win)
SAC(loss) vs MIN( win)
SAS( win) vs NOP(loss)
0.4444444444444444
DET(loss) vs MIL( win)
HOU(loss) vs DAL( win)
LAC( win) vs UTA(loss)
MEM(loss) vs WAS( win)
MIA(loss) vs SAS( win)
OKC( win) vs LAL(loss)
PHO(loss) vs GSW( win)
0.571

KeyboardInterrupt: 

# predict winner per game

In [23]:
com = [['MIL', 'BOS'], ['ATL', 'DAL'], ['CHO', 'DET'], ['BRK', 'IND'], ['NOP', 'MEM'], ['MIA', 'ORL'], ['POR', 'PHO'], ['HOU', 'SAC'], ['MIN', 'SAS'], ['DEN', 'UTA'], ['PHI', 'WAS']]
for i in com:
    p = predict(i[0], i[1])
    if p == 1:
        print('%s( win) vs %s(loss)' % (i[0], i[1]))
    else:
        print('%s(loss) vs %s( win)' % (i[0], i[1]))

MIL( win) vs BOS(loss)
ATL( win) vs DAL(loss)
CHO(loss) vs DET( win)
BRK(loss) vs IND( win)
NOP(loss) vs MEM( win)
MIA( win) vs ORL(loss)
POR( win) vs PHO(loss)
HOU(loss) vs SAC( win)
MIN(loss) vs SAS( win)
DEN(loss) vs UTA( win)
PHI(loss) vs WAS( win)


In [296]:
dfdate

,Date,Visitor/Neutral,PTS,Home/Neutral,PTS.1
0,2016-10-25,NYK,88,CLE,117
1,2016-10-25,SAS,129,GSW,100
2,2016-10-25,UTA,104,POR,113


In [4]:
df.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\trainingDataSet.xlsx')

# test

In [17]:
model.trainable_weights

[<tf.Variable 'dense_1/kernel:0' shape=(18, 60) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(60,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(60, 30) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(30,) dtype=float32_ref>,
 <tf.Variable 'dense_3/kernel:0' shape=(30, 1) dtype=float32_ref>,
 <tf.Variable 'dense_3/bias:0' shape=(1,) dtype=float32_ref>]

In [16]:
model.get_weights()

[array([[-0.09973076, -0.28036681,  0.41126528, ..., -0.37119246,
         -0.18673807, -0.02743476],
        [ 0.23101056,  0.18751974, -0.15770242, ...,  0.26227781,
          0.11880766, -0.25534803],
        [-0.21106575, -0.01742065,  0.16644265, ..., -0.00496429,
         -0.26998398, -0.14453615],
        ..., 
        [-0.03454875, -0.04719381,  0.0329175 , ...,  0.26350275,
          0.15770875,  0.18335274],
        [ 0.1076322 , -0.07261449,  0.11019521, ...,  0.15897585,
          0.36988306,  0.21668501],
        [ 0.17670396, -0.52645206,  0.15367696, ..., -0.40337008,
         -0.04699036, -0.06610261]], dtype=float32),
 array([-0.27998814,  0.24896714, -0.06985473, -0.13744625, -0.30568719,
         0.33496395,  0.31904963,  0.15952404,  0.16471496,  0.25016752,
        -0.1061762 ,  0.10702935, -0.31374237, -0.06228459, -0.06802844,
        -0.02397024,  0.10124181,  0.15716448, -0.08572163,  0.01312252,
        -0.0512724 ,  0.04232288,  0.19391552,  0.32137033,  0.19

In [22]:
#測試讀檔寫檔
df = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\error_test.xlsx')
dfsli = df.tail(1)
for i in range(10):
    df = df.append(dfsli, ignore_index=True)
df.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\error_test.xlsx')

In [2]:
df = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\2017\\East\\ATL\\home_away.xlsx')

In [11]:
a = {u'ast': 2, 'win': 1.0, u'fbptsa': 6, u'tf': 1, u'bpts': -4, 'away': u'LAC', u'pip': -2, 'home': u'CHA', u'dreb': 4, u'fga': 4, u'tmtov': 0, u'scp': 14, 'date': u'2017-11-19', u'fbptsm': 5, u'tpa': -3, u'fgm': 1, u'stl': 2, u'fbpts': 10, u'ble': 13, u'tov': -6, u'oreb': 1, u'potov': 16, u'fta': 10, u'pipm': -1, u'pf': -6, u'tmreb': -2, u'blk': 3, u'reb': 5, u'pipa': -4, u'ftm': 10, u'tpm': 3}